In [7]:
import sys
import nltk
import torch.nn.functional as F
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
sys.path.insert(0, '..')
import pickle
from torch.autograd import grad as torch_grad



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
print(torch.__version__)
print("GPU Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = "cpu"

2.0.1
GPU Available: False


In [9]:
batch_size = 64
word_embedding_dim = 50
output_shape = (1, 105, 8)
torch.manual_seed(1)
np.random.seed(1)

In [10]:

# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs_Sentence.pkl", 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [11]:
def create_word_label_embeddings_sentence(Word_Labels_List, EEG_word_level_embeddings):

    tokenized_words = []
    for i in range(len(Word_Labels_List)):
        tokenized_words.append([Word_Labels_List[i]])
    model = Word2Vec(sentences=tokenized_words, vector_size=word_embedding_dim, window=5, min_count=1, workers=4)
    word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}

    list_of_sentences = []
    sentence = []

    max_sentence_length = 0
    for words in range (0, len(EEG_word_level_labels)):

        current_word = EEG_word_level_labels[words]
        current_word_embedding = word_embeddings[current_word]

        if words == len(EEG_word_level_labels)-1:
            if len(sentence) > max_sentence_length:
                max_sentence_length = len(sentence)
            list_of_sentences.append(sentence)

        elif current_word == "SOS" and sentence != []:
            if len(sentence) > max_sentence_length:
                max_sentence_length = len(sentence)

            list_of_sentences.append(sentence)
            sentence = []
        elif current_word != "SOS":
            sentence.append(current_word_embedding)



    EEG_sentence_list = []
    index_counter = 0
    for sentence in list_of_sentences:
        EEG_sentence = EEG_word_level_embeddings[index_counter:index_counter+len(sentence)]
        for i in range(max_sentence_length-len(EEG_sentence)):
            EEG_sentence.append(np.zeros((105, 8)))

        EEG_sentence_list.append(EEG_sentence)
        index_counter += len(sentence)
        if len(sentence) > max_sentence_length:
            max_sentence_length = len(sentence)

    for sentence in list_of_sentences:
        for i in range(max_sentence_length-len(sentence)):
            sentence.append(np.zeros((word_embedding_dim)))

    for i in range(len(EEG_sentence_list)):
        EEG_sentence = EEG_sentence_list[i]
        contact_EEG_sentence = np.concatenate(EEG_sentence, axis=1)
        EEG_sentence_list[i] = contact_EEG_sentence

        word_embedding_sentence = list_of_sentences[i]
        concat_word_embedding_sentence = np.concatenate(word_embedding_sentence, axis=0)
        list_of_sentences[i] = concat_word_embedding_sentence

    return EEG_sentence_list, list_of_sentences


def create_dataloader_sentence(EEG_word_level_embeddings, Embedded_Word_labels):
    #EEG_word_level_embeddings_normalize = (EEG_word_level_embeddings - np.mean(EEG_word_level_embeddings)) / np.std(EEG_word_level_embeddings)
    #Assuming EEG_synthetic is the generated synthetic EEG data
    #EEG_synthetic_denormalized = (EEG_synthetic * np.max(np.abs(EEG_word_level_embeddings))) + np.mean(EEG_word_level_embeddings)

    EEG_word_level_embeddings_normalize = (EEG_word_level_embeddings - np.mean(EEG_word_level_embeddings)) / np.max(np.abs(EEG_word_level_embeddings))


    float_tensor = torch.tensor(EEG_word_level_embeddings_normalize, dtype=torch.float32)
    float_tensor = float_tensor.unsqueeze(1)

    #print(EEG_word_level_embeddings_normalize)
    # Calculate mean and standard deviation
    print(torch.isnan(float_tensor).any())


    Embedded_Word_labels = torch.tensor(Embedded_Word_labels, dtype=torch.float32)

    train_data = []
    for i in range(len(float_tensor)):
       train_data.append([float_tensor[i], Embedded_Word_labels[i]])
    trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)
    return trainloader



In [12]:
EEG_sentence_list, list_of_sentences = create_word_label_embeddings_sentence(EEG_word_level_labels, EEG_word_level_embeddings)
trainloader = create_dataloader_sentence(EEG_sentence_list, list_of_sentences)

tensor(False)


C:\Users\gxb18167\AppData\Local\Temp\ipykernel_21048\2386319609.py:78: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\b\abs_abjetg6_iu\croot\pytorch_1686932924616\work\torch\csrc\utils\tensor_new.cpp:248.)
  Embedded_Word_labels = torch.tensor(Embedded_Word_labels, dtype=torch.float32)


In [13]:
#sanity check:
i1, l1 = next(iter(trainloader))
print(i1.shape, l1)

torch.Size([64, 1, 105, 456]) tensor([[-0.0020, -0.0195, -0.0014,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0200,  0.0011, -0.0103,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0038,  0.0167,  0.0171,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0165,  0.0192, -0.0089,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0197, -0.0079,  0.0132,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0148, -0.0035, -0.0046,  ...,  0.0000,  0.0000,  0.0000]])


In [14]:
z_size = 100
image_size = (105, 8)

n_filters = 32

In [15]:
## Loss function and optimizers:
loss_fn = nn.BCELoss()

In [16]:
def create_noise(batch_size, z_size, mode_z):
    if mode_z == 'uniform':
        input_z = torch.rand(batch_size, z_size)*2 - 1
    elif mode_z == 'normal':
        input_z = torch.randn(batch_size, z_size)
    return input_z

mode_z = 'uniform'
fixed_z = create_noise(batch_size, z_size, mode_z).to(device)

def create_samples(g_model, input_z, input_t):
    g_output = g_model(input_z, input_t)
    images = torch.reshape(g_output, (batch_size, *image_size))
    return (images+1)/2.0

In [17]:
noise = create_noise(64, 100, "uniform")

In [18]:
noise.shape

torch.Size([64, 100])

In [19]:
class Generator(nn.Module):
    def __init__(self, noise_dim, word_embedding_dim):
        super(Generator, self).__init__()

        self.noise_dim = noise_dim
        self.word_embedding_dim = word_embedding_dim

        # Define the layers of your generator
        self.fc_noise = nn.Linear(noise_dim, 105 * 456)
        self.fc_word_embedding = nn.Linear(word_embedding_dim, 105 * 456)
        self.conv1 = nn.Conv2d(2, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)



    def forward(self, noise, word_embedding):
        # Process noise
        noise = self.fc_noise(noise)
        noise = noise.view(noise.size(0), 1, 105, 456)

        # Process word embedding
        word_embedding = self.fc_word_embedding(word_embedding.to(device))
        word_embedding = word_embedding.view(word_embedding.size(0), 1, 105, 456)

        print("word_embedding:", word_embedding.shape)
        print("input:", noise.shape)

        # Concatenate noise and word embedding
        combined_input = torch.cat([noise, word_embedding], dim=1)

        print("combined_input:", combined_input.shape)

        # Upsample and generate the output
        z = self.conv1(combined_input)
        z = self.bn1(z)
        z = self.relu(z)
        z = self.conv2(z)

        return z

class DiscriminatorWGAN(nn.Module):
    def __init__(self, n_filters, word_embedding_dim):
        super(DiscriminatorWGAN, self).__init__()

        self.word_embedding_dim = word_embedding_dim
        self.fc_word_embedding = nn.Linear(word_embedding_dim, 105 * 456)

        self.network = nn.Sequential(
            nn.Conv2d(2, n_filters, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters, n_filters*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(n_filters * 2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters*2, n_filters*4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(n_filters*4),
            nn.LeakyReLU(0.2),

            nn.Flatten(),  # Flatten spatial dimensions

            # Fully connected layer to reduce to a single value per sample
            nn.Linear(n_filters*4 * (105 // 8) * (456 // 8), 1),
            nn.Sigmoid()
        )

    def forward(self, input, word_embedding):
        word_embedding = self.fc_word_embedding(word_embedding.to(device))
        word_embedding = word_embedding.view(word_embedding.size(0), 1, 105, 456)


        combined_input = torch.cat([input, word_embedding], dim=1)

        output = self.network(combined_input)
        return output


In [24]:
gen_model = Generator(z_size, 2850).to(device)
disc_model = DiscriminatorWGAN(n_filters, 2850).to(device)

g_optimizer = torch.optim.Adam(gen_model.parameters(), 0.00002)
d_optimizer = torch.optim.Adam(disc_model.parameters(), 0.00002)

In [25]:

def gradient_penalty(real_data, generated_data, input_t):
    batch_size = real_data.size(0)

    # Calculate interpolation
    alpha = torch.rand(real_data.shape[0], 1, 1, 1, requires_grad=True, device=device)
    #print("Gen:", generated_data.shape)
    interpolated = alpha * real_data + (1 - alpha) * generated_data

    # Calculate probability of interpolated examples
    proba_interpolated = disc_model(interpolated, input_t)

    # Calculate gradients of probabilities with respect to examples
    gradients = torch_grad(outputs=proba_interpolated, inputs=interpolated,
                           grad_outputs=torch.ones(proba_interpolated.size(), device=device),
                           create_graph=True, retain_graph=True)[0]

    gradients = gradients.view(batch_size, -1)
    gradients_norm = gradients.norm(2, dim=1)
    return lambda_gp * ((gradients_norm - 1)**2).mean()


In [26]:
## Train the discriminator
def d_train_wgan(x, input_t):

    #Init gradient
    disc_model.zero_grad()


    batch_size = x.size(0)
    x = x.to(device)

    #Discriminating Real Images
    d_real = disc_model(x, input_t)

    #Building Z
    input_z = create_noise(batch_size, z_size, mode_z).to(device)
    #Building Fake T
    fake_t = torch.zeros(batch_size, 2850).to(device)

    #Generating Fake Images
    g_output = gen_model(input_z, fake_t)

    #Discriminating Fake Images
    d_generated = disc_model(g_output, input_t)

    d_loss = d_generated.mean() - d_real.mean() + gradient_penalty(x.data, g_output.data, input_t)

    d_loss.backward()
    d_optimizer.step()

    return d_loss.data.item()

In [27]:
## Train the generator
def g_train_wgan(x, input_t):

    #Init gradient
    gen_model.zero_grad()

    #Building Z
    batch_size = x.size(0)
    input_z = create_noise(batch_size, z_size, mode_z).to(device)

    #Building Fake T
    fake_t = torch.zeros(batch_size, 2850).to(device)

    #Generating Fake Images
    g_output = gen_model(input_z, fake_t)

    #Discriminating Fake Images
    d_generated = disc_model(g_output, fake_t)
    g_loss = -d_generated.mean()
    #print("G Loss:", g_loss)

    # gradient backprop & optimize ONLY G's parameters
    g_loss.backward()
    g_optimizer.step()

    return g_loss.data.item()

In [28]:
epoch_samples_wgan = []
lambda_gp = 10.0
num_epochs = 100
torch.manual_seed(1)
critic_iterations = 5
save_interval = 5
checkpoint_path = 'Textual_WGAN_GP_checkpoint_epoch_{}.pth'
final_model_path = 'Textual_WGAN_GP_model_final.pth'

for epoch in range(1, num_epochs+1):
    gen_model.train()
    d_losses, g_losses = [], []
    for i, (x, t) in enumerate(trainloader):
        #print("T:", t.shape)
        for _ in range(critic_iterations):
            d_loss = d_train_wgan(x, t)
            print("D Loss:", d_loss)
        d_losses.append(d_loss)
        g_losses.append(g_train_wgan(x, t))

    print(f'Epoch {epoch:03d} | D Loss >>'
          f' {torch.FloatTensor(d_losses).mean():.4f}')
    print(f'Epoch {epoch:03d} | G Loss >>'
          f' {torch.FloatTensor(g_losses).mean():.4f}')

        # Save checkpoints at regular intervals
    if epoch % save_interval == 0:
        torch.save({
            'epoch': epoch,
            'gen_model_state_dict': gen_model.state_dict(),
            'optimizer_state_dict': g_optimizer.state_dict(),
            'd_losses': d_losses,
            'g_losses': g_losses,
        }, checkpoint_path.format(epoch))


    '''
    gen_model.eval()
    epoch_samples_wgan.append(
        create_samples(gen_model, fixed_z, t).detach().cpu().numpy())
    '''

word_embedding: torch.Size([64, 1, 105, 456])
input: torch.Size([64, 1, 105, 456])
combined_input: torch.Size([64, 2, 105, 456])
D Loss: 15.738903999328613
word_embedding: torch.Size([64, 1, 105, 456])
input: torch.Size([64, 1, 105, 456])
combined_input: torch.Size([64, 2, 105, 456])
D Loss: 19.257532119750977



KeyboardInterrupt

